In [ ]:
%matplotlib inline

In [ ]:
import ifc_utils
import keras
import numpy
import matplotlib.pyplot
import os.path
import pandas

In [ ]:
directory = "data/processed"

data = {
    "abnormal": "data/raw/abnormal",
    "normal": "data/raw/normal"
}

channels = [0, 2, 5, 6, 11]

split = 0.8

In [ ]:
ifc_utils.parse(directory, data, channels)

ifc_utils.split(directory, data.keys(), split)

class_weights = ifc_utils.class_weights(directory, data)

In [ ]:
shape = (32, 32, 1)

x = keras.layers.Input(shape)

In [ ]:
classes = 2

The classifier architecture is adapted from the LeNet-5 convolutional neural network by Yann LeCunn, et al.

In [ ]:
options = {"activation": "relu", "kernel_size": (3, 3)}

y = keras.layers.Conv2D(32, **options)(x)
y = keras.layers.Conv2D(64, **options)(y)

y = keras.layers.MaxPooling2D()(y)

y = keras.layers.Dropout(0.5)(y)

y = keras.layers.Flatten()(y)

y = keras.layers.Dense(128, activation="relu")(y)

y = keras.layers.Dropout(0.5)(y)

y = keras.layers.Dense(classes)(y)

y = keras.layers.Activation("softmax")(y)

In [ ]:
model = keras.models.Model(x, y)

In [ ]:
loss = keras.losses.categorical_crossentropy

optimizer = keras.optimizers.Adam(0.0001)

model.compile(
    loss=loss, 
    metrics=[
        "accuracy"
    ],
    optimizer=optimizer
)

In [ ]:
training_x = numpy.load(os.path.join(directory, "training_x.npy"))

training_y = numpy.load(os.path.join(directory, "training_y.npy"))

In [ ]:
csv_logger = keras.callbacks.CSVLogger("training.csv")

early_stopping = keras.callbacks.EarlyStopping(patience=32)

model_checkpoint = keras.callbacks.ModelCheckpoint("checkpoint.hdf5")

reduce_learning_rate_on_plateau = keras.callbacks.ReduceLROnPlateau()

In [ ]:
options = {
    "batch_size": 4096,
    "class_weight": class_weights,
    "callbacks": [
        csv_logger,
        early_stopping,
        model_checkpoint,
        reduce_learning_rate_on_plateau
    ],
    "epochs": 256,
    "validation_split": 0.25,
    "verbose": 0,
    "x": training_x,
    "y": training_y    
}

%time model.fit(**options)

In [ ]:
observations = pandas.read_csv("training.csv")

_, (a, b) = matplotlib.pyplot.subplots(nrows=1, ncols=2, figsize=(12, 4))

a.plot(observations["acc"], "b")
a.plot(observations["val_acc"], "r")

b.plot(observations["loss"], "b")
b.plot(observations["val_loss"], "r")

In [ ]:
model.load_weights("./checkpoint.hdf5")

In [ ]:
test_x = numpy.load(os.path.join(directory, "test_x.npy"))

test_y = numpy.load(os.path.join(directory, "test_y.npy"))

In [ ]:
model.evaluate(test_x, test_y)